In [1]:
import pandas as pd
from surprise import Dataset, Reader
from surprise import KNNBasic
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise.model_selection import GridSearchCV
from collections import defaultdict
from sklearn.metrics import precision_score, recall_score

In [2]:
# Đọc dữ liệu
review_data = pd.read_csv(r"C:\Users\anhn2\Documents\DJANGO\DA\TIKI\comments_data.csv")

# Chuẩn bị dữ liệu cho Surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(review_data[['user_id', 'product_id', 'rating']], reader)

In [3]:
# Chia dữ liệu thành tập huấn luyện và kiểm tra
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

In [4]:
# Cấu hình các tham số cho GridSearch
param_grid = {
    'sim_options': {
        'name': ['cosine', 'pearson'],  # Phương pháp tính tương đồng
        'user_based': [True, False]  # User-User hoặc Item-Item
    },
    'k': [10, 15, 20, 30, 50]  # Số lượng hàng xóm
}

# Khởi tạo GridSearch với random_state cố định
grid_search = GridSearchCV(KNNBasic, param_grid, measures=['rmse'], cv=5)
grid_search.fit(data)

# Lưu kết quả của từng bộ tham số vào DataFrame
results = []
for params, mean_rmse in zip(grid_search.cv_results['params'], grid_search.cv_results['mean_test_rmse']):
    results.append({
        'similarity_metric': params['sim_options']['name'],
        'user_based': params['sim_options']['user_based'],
        'k': params['k'],
        'mean_rmse': mean_rmse
    })

results_df = pd.DataFrame(results)
results_df

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing th

,similarity_metric,user_based,k,mean_rmse
0,cosine,True,10,1.061173
1,cosine,True,15,1.061084
2,cosine,True,20,1.061030
3,cosine,True,30,1.061013
4,cosine,True,50,1.061008
5,cosine,False,10,1.050711
6,cosine,False,15,1.050707
7,cosine,False,20,1.050704
8,cosine,False,30,1.050697
9,cosine,False,50,1.050696


In [5]:
best_params = grid_search.best_params['rmse']
best_rmse = grid_search.best_score['rmse']

# Lựa chọn tham số tốt nhất và in chi tiết
best_result = results_df.loc[
    (results_df['similarity_metric'] == best_params['sim_options']['name']) &
    (results_df['user_based'] == best_params['sim_options']['user_based']) &
    (results_df['k'] == best_params['k'])
]

print("\n--- Tham số tốt nhất---")
print(best_result)


--- Tham số tốt nhất---
  similarity_metric  user_based   k  mean_rmse
9            cosine       False  50   1.050696


In [6]:
best_params_row = results_df.loc[results_df['mean_rmse'].idxmin()]
best_params = best_params_row.to_dict()
best_sim_options = {
    'name': best_params['similarity_metric'],
    'user_based': best_params['user_based']
}

best_k = best_params['k']

# Khởi tạo mô hình với tham số tốt nhất
best_algo = KNNBasic(sim_options=best_sim_options, k=best_k)
best_algo.fit(trainset)
predictions = best_algo.test(testset)

# Đánh giá trên tập kiểm tra
final_rmse = accuracy.rmse(predictions)
print(f"Final RMSE on test set: {final_rmse}")

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.0414
Final RMSE on test set: 1.0413686749026825


In [7]:
# Hàm lấy Top-K sản phẩm được gợi ý cho mỗi người dùng
def get_top_k(predictions, k=10):
    top_k_recommendations = defaultdict(list)
    for prediction in predictions:
        uid = prediction.uid
        iid = prediction.iid
        est = prediction.est
        top_k_recommendations[uid].append((iid, est))
    # Sắp xếp theo giá trị dự đoán (est) giảm dần và lấy top K
    for uid, user_ratings in top_k_recommendations.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_k_recommendations[uid] = user_ratings[:k]
    return top_k_recommendations

# Precision@K và Recall@K
def precision_recall_at_k(predictions, k=10, threshold=3.5):
    # Lấy Top-K gợi ý
    top_k = get_top_k(predictions, k)
    precisions = []
    recalls = []

    for uid, user_ratings in top_k.items():
        # Các sản phẩm thực sự liên quan (n_rel)
        n_rel = sum((pred.r_ui >= threshold) for pred in predictions if pred.uid == uid)
        
        # Các sản phẩm được gợi ý trong Top-K (n_rec_k)
        n_rec_k = sum((est >= threshold) for (_, est) in user_ratings)
        
        # Các sản phẩm vừa liên quan vừa nằm trong Top-K (n_rel_and_rec_k)
        n_rel_and_rec_k = sum(
            ((pred.r_ui >= threshold) and (pred.est >= threshold))
            for pred in predictions
            if pred.uid == uid and any(iid == pred.iid for (iid, _) in user_ratings)
        )

        # Precision và Recall
        precision = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0
        recall = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

        precisions.append(precision)
        recalls.append(recall)

    # Trung bình Precision và Recall
    avg_precision = sum(precisions) / len(precisions) if precisions else 0
    avg_recall = sum(recalls) / len(recalls) if recalls else 0
    return avg_precision, avg_recall

# Tính toán các giá trị Precision@K và Recall@K
precision_at_k, recall_at_k = precision_recall_at_k(predictions, k=10)
f1_score = 2 * (precision_at_k * recall_at_k) / (precision_at_k + recall_at_k) if (precision_at_k + recall_at_k) != 0 else 0

# In kết quả
print(f"Precision@10: {precision_at_k:.4f}")
print(f"Recall@10: {recall_at_k:.4f}")
print(f"F1-Score@10: {f1_score:.4f}")

Precision@10: 0.8490
Recall@10: 0.8512
F1-Score@10: 0.8501


In [8]:
import joblib

# Lưu mô hình đã huấn luyện
joblib.dump(best_algo, 'knn_model.pkl')
print("Mô hình đã được lưu thành công.")


Mô hình đã được lưu thành công.


In [9]:
import time

# Đo thời gian huấn luyện
start_time = time.time()

# Huấn luyện mô hình với tham số tốt nhất
best_algo.fit(trainset)

# Kết thúc đo thời gian
end_time = time.time()

# Tính thời gian huấn luyện
training_time = end_time - start_time
print(f"Thời gian huấn luyện mô hình: {training_time:.2f} giây")

# Lưu mô hình đã huấn luyện
import joblib
joblib.dump(best_algo, 'knn_model.pkl')
print("Mô hình đã được lưu thành công.")


Computing the cosine similarity matrix...
Done computing similarity matrix.
Thời gian huấn luyện mô hình: 0.01 giây
Mô hình đã được lưu thành công.
